# Alex Lee, Kaeley Taylor, Yue Hu
# Geog 458 Final Project
### This program takes in a test csv file of the King County EXTR_Parcel file. It takes housing prices of various land parcels from the file and calculates percent change of housing prices from 2005 to 2010 and 2010 to 2015. For each of the cities that are listed, a geojson is created with the percent changes as attributes.

In [7]:
import urllib2
import lxml
from lxml import html
import xml.etree.ElementTree as E
import csv
import geopandas
import pandas
import json
from osgeo import ogr

In [8]:
parcel_number = []

# Open and read the CSV file that contains parcel numbers and city of land parcels
#file = open("U:\\Geog 458\\test.csv")
file = open("U:\\Geog 458\\test2med.csv")
# file = open("U:\\Geog 458\\EXTR_Parcel.csv")
reader = csv.reader(file)
city_set = set()
for line in reader:
    # Format the land parcels to fit the parameters for the King County Assessment website
    if len(line[0]) < 6:
        for i in range(6 - len(line[0])):
            line[0] = str('0' + line[0]) 
    if len(line[1]) < 4:
        for i in range(4 - len(line[1])):
            line[1] = str('0' + line[1])
    if line[15] not in city_set:
        city_set.add(line[15])
    parcel_number.append([line[0] + line[1], line[15]])


three_year_prices = []
count = 0

three_year_prices = []
count = 0

In [11]:
for parcel in parcel_number:
    # Open a new request for each parcel number
    url = "http://blue.kingcounty.com/Assessor/eRealProperty/Dashboard.aspx?ParcelNbr=" + parcel[0]
    con = urllib2.urlopen(url)
    doc_text = con.read()
    doc = lxml.html.fromstring(doc_text)

    prices = []
    for row in doc.cssselect("body #kingcounty_gov_cphContent_GridViewDBTaxRoll"):
        for li in row.cssselect("tr.GridViewRowStyle td:nth-child(8)"):
            # Extract data from the table on the website
            prices.append(li.text_content())
    # Save parcel appraisal from 2015, 2010, and 2005
    if len(prices):
        try:
            three_year_prices.append([parcel[1]])
            three_year_prices[count].append(prices[1].replace(',', ''))
            three_year_prices[count].append(prices[3].replace(',', ''))
            three_year_prices[count].append(prices[6].replace(',', ''))
        except IndexError:
            continue
    count += 1

In [12]:
index_count = 0
# Save percent change of housing prices in dictionaries
cities_15_10_change = {}
cities_10_05_change = {}

for parcel in three_year_prices:
    print parcel
    # Store change of housing price results in corresponding dictionary
    if len(parcel) < 4 or parcel[1] == '0' or parcel[2] == '0' or parcel[3] == '0':
        del parcel
    else:
        if parcel[0] not in cities_15_10_change.keys():
            cities_15_10_change[parcel[0]] = float(parcel[1]) / float(parcel[2])
            cities_10_05_change[parcel[0]] = float(parcel[2]) / float(parcel[3])
        else:
            cities_15_10_change[parcel[0]] = (cities_15_10_change[parcel[0]] + (float(parcel[1]) / float(parcel[2]))) / 2
            cities_10_05_change[parcel[0]] = (cities_10_05_change[parcel[0]] + (float(parcel[2]) / float(parcel[3]))) / 2
    index_count += 1

['KENT']
['SEATTLE', '1235000', '1027000', '797000']
['SEATTLE', '344000', '344000', '304000']
['SAMMAMISH', '1000', '1000', '1000']
['SEATTLE', '350000', '330000', '302000']
['KENT', '296000', '244000', '254000']
['RENTON']
['SEATTLE', '670000', '606000', '481000']
['KENT', '230000', '206000', '189000']
['SEATTLE', '429000', '375000', '314000']
['KIRKLAND', '490000', '379000', '326000']
['AUBURN']
['RENTON', '639000', '535000', '562000']
['SAMMAMISH', '2671000', '2187000', '680000']
['KING COUNTY', '596000', '498000', '434000']
['FEDERAL WAY', '138000', '135000', '112000']
['SEATTLE', '183000', '188000', '170000']
['SEATTLE', '410000', '361000', '299000']
['SEATTLE', '393000', '384000', '284000']
['REDMOND', '381000', '278000', '260000']
['DES MOINES', '138000', '138000', '145000']
['KING COUNTY', '3267', '3606', '3630']
['KING COUNTY', '264000', '223000', '210000']
['SEATTLE', '207000', '218000', '177000']
['BELLEVUE', '768000', '532000', '421000']
['SEATTLE', '947000', '734000', '28

In [15]:
for city in city_set:
    try:
        # Access API to get a list of coordinates for a polygon of each city
        json_text = urllib2.urlopen('http://nominatim.openstreetmap.org/search?q=' + city + '&state=washington&country=USA&format=json&polygon=1').read()
        json_text = json.loads(json_text)
        # Skip errors for sake of time
        try:
            polypoints = json_text[0]['polygonpoints']
        except KeyError:
            polypoints = json_text[1]['polygonpoints']
        except IndexError:
            continue
        points_list = []
        for point in polypoints:
            points_list.append([float(str(point[0])), float(str(point[1]))])

        # Create the polygon
        ring = ogr.Geometry(ogr.wkbLinearRing)
        for point in points_list:
            ring.AddPoint(point[0], point[1])

        poly = ogr.Geometry(ogr.wkbPolygon)
        poly.AddGeometry(ring)

        # Write the geojson to a file
        geojson_file = open(city + ".geojson", 'w')
        geojson_file.write(poly.ExportToJson())
        geojson_file.close()

        # Add the attributes of percent change for corresponding year
        gdf = geopandas.read_file(city + ".geojson")
        gdf['15_10'] = cities_15_10_change[city]
        gdf['10_05'] = cities_10_05_change[city]
        gdf.to_file("modified" + city + ".geojson",driver="GeoJSON")
    # Skip errors
    except ValueError:
        continue
    except KeyError:
        continue